<a href="https://colab.research.google.com/github/Winzen/mides-rascunho/blob/main/code/scraping/se/%5Bse%5Dmapear_empenho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Constantes

In [ ]:
path_drive_input = None # Onde será extraido e salvo as paginas de empenho
path_drive_csv = None # Onde será extraido e salvo o csv das entidades
path_drive_csv_mapeamento = # Onde será extraido e salvo os CSVs do mapeamento dos empenhos

# Importação

In [ ]:
import requests
import os
import csv
import shutil
import glob
import concurrent.futures
import datetime as dt
import time
import re
import pandas as pd
from time import sleep
from more_itertools import batched
from bs4 import BeautifulSoup, element
from urllib3 import PoolManager, disable_warnings, exceptions, Timeout
from urllib.parse import urlencode
from zipfile import ZipFile


def send_folder_drive(path_drive: str, path_input: str) -> None:

  if not len(os.listdir(path_input)) > 0:
    raise Exception("Essa Pasta vazia")
    return None

  shutil.make_archive(path_drive,
                      'zip',
                      path_input)


def make_list_chunks(values: list, chunk_size:int = 2000) -> list:
  resquet_in_chunks = [chunk for chunk in batched(values, chunk_size)]
  return resquet_in_chunks


def extrair_zip(path, path_output) -> None:
  # loading the temp.zip and creating a zip object
  with ZipFile(path, 'r') as zObject:

      # Extracting all the members of the zip
      # into a specific location.
      zObject.extractall(
          path=path_output)


def extrair(path_himl: str) -> list|str:

  try:

    html = ler(path_himl)
    soup = BeautifulSoup(html, "html.parser")
    basic_info = path_himl.split("/")[3:-1]
    linhas_table = soup.select("table#ctl00_ContentPlaceHolder1_grdEmpenhos tr")[3:-3] # Seleciona linhas da tag Table e remove desnecessarios
    linhas = [get_row_values(row, basic_info) for row in linhas_table]

    return linhas

  except:

    return path_himl


def ler(path: str) -> str:

  with open(path, "r") as slot_read:
    return slot_read.read()


def get_row_values(row: element.Tag, info_basic: list) -> list:

  colunas = row.find_all("td")
  coluna_link = colunas.pop()
  valores = [coluna.text.strip() for coluna in colunas]

  parametros = coluna_link.select_one("a.thickbox").get("href").replace(" ", "")
  link = "https://www.tcese.tc.br/portaldatransparencia/" + parametros
  valores.append(link)

  linha = info_basic + valores

  return linha

# Extrair CSV

In [ ]:
extrair_zip(path=path_drive_csv + ".zip",
        path_output="/content/csvs")

In [ ]:
extrair_zip(path=path_drive_input + ".zip",
        path_output="/content/input")

# Mapear Empenhos

In [ ]:
path_csvs = "/content/input/**/**/**/*.html"
htmls = glob.glob(path_csvs)

with concurrent.futures.ThreadPoolExecutor() as executor:
  tasks = [task for task in executor.map(extrair, htmls)]
  linhas_paginas = [task for task in tasks if type(task) != str]
  erros = [task for task in tasks if type(task) == str]

print(f"De {len(htmls)} paginas foram coletadas {len(linhas_paginas)} com sucesso")

linhas = [linha for linhas in linhas_paginas for linha in linhas]

colunas = ["ano", "municipio", "unidade",
           "numero_empenho", "data_empenho", "participante",
           "valor_empenhado", "valor_pago", "link"]

df = pd.DataFrame(linhas, columns=colunas)
df["coletado"] = False
df.to_csv("csvs/se_mapeamentos_empenhos.csv", index=False)

## Dividir por anos

In [ ]:
anos = df.ano.unique()
for ano in anos:
  df_ano = df[df.ano == ano]
  df_ano.to_csv(f"csvs/se_mapeamentos_empenhos_{ano}.csv", index=False)

# Mandar no drive

In [ ]:
drive.mount('/content/drive')

send_folder_drive(path_drive_csv_mapeamento,
                "/content/csvs")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
